In [69]:

import cv2
import numpy as np
import tensorflow as tf
from npeet import entropy_estimators
from tensorflow_wavelets.utils.models import DWT
from matplotlib import pyplot as plt
from sklearn.metrics import mutual_info_score


In [70]:
# git clone https://github.com/gregversteeg/NPEET.git
# cd NPEET
# pip install .
# x = [[1.3],[3.7],[5.1],[2.4],[3.4]]
# y = [[1.5],[3.32],[5.3],[2.3],[3.3]]
# entropy_estimators.mi(x,y)
# Out: 0.168

In [71]:

def mutual_info_wavelet(wavelet_name, img, shape=(240, 240)):
    # resize input image to be square
    img = cv2.resize(img, shape)
    # expand dims for the dwt transfor
    img_t = np.expand_dims(img, axis=-1)
    img_ex = np.expand_dims(img_t, axis=0)
    # concat=1 LL,LH,HL,HH in one image
    coeffs = DWT.DWT(wavelet_name, 1, "VALID", "SYMMETRIC")(tf.dtypes.cast(img_ex, tf.float32))
    # extract resault to numpy
    dtw_img = coeffs[0,:, :, 0].numpy()
    # resize the output to be like the input (maybe we need to use crop?)
    dtw_img_res = cv2.resize(dtw_img, dsize=img.shape, interpolation=cv2.INTER_LINEAR)

    flat_img = img_ex.flatten()
    flat_coeffs = dtw_img_res.flatten()
    mi = entropy_estimators.mi(flat_img, flat_coeffs)
    print(f"Mutual Information = {mi}, for {wavelet_name} Wavelet")



In [ ]:
input = tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, padding='same')(tf.dtypes.cast(input, tf.float32))

In [72]:
img = cv2.imread("../src/input/Lenna_orig.png", 0)
mutual_info_wavelet("haar", img)
mutual_info_wavelet("db2", img)
mutual_info_wavelet("db3", img)
mutual_info_wavelet("sym3", img)
mutual_info_wavelet("bior1.3", img)

Mutual Information = 0.30492119228714604, for haar Wavelet
Mutual Information = 0.24842043893401106, for db2 Wavelet
Mutual Information = 0.23992783265615508, for db3 Wavelet
Mutual Information = 0.2402050299189054, for sym3 Wavelet
Mutual Information = 0.24801498672352387, for bior1.3 Wavelet
